<a href="https://colab.research.google.com/github/iwatake2222/colaboratory_study/blob/master/object_detection/beer_detection/beer_detection_train_faster_rcnn_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object detection model training
- dataset: beer
- pretrained model: faster_rcnn_resnet101_coco

## Preparation

In [0]:
from google.colab import drive 
drive.mount('/content/drive')
!ls

TRAIN_DIR = "/content/drive/My Drive/training/beer_detection/faster_rcnn_resnet101_coco_11_06_2017_20191230"

In [4]:
%tensorflow_version 1.x
# restart runtime
# !pip install tensorflow
import tensorflow as tf
print(tf.__version__)
!pip install pycocotools
!pip list

!git clone --depth 1 https://github.com/tensorflow/models
%cd models/research/
!protoc object_detection/protos/*.proto --python_out=.
!pip install .
%cd /content

1.15.0
Package                  Version    
------------------------ -----------
absl-py                  0.8.1      
alabaster                0.7.12     
albumentations           0.1.12     
altair                   4.0.0      
asgiref                  3.2.3      
astor                    0.8.1      
astropy                  3.0.5      
atari-py                 0.2.6      
atomicwrites             1.3.0      
attrs                    19.3.0     
audioread                2.1.8      
autograd                 1.3        
Babel                    2.7.0      
backcall                 0.1.0      
backports.tempfile       1.0        
backports.weakref        1.0.post1  
beautifulsoup4           4.6.3      
bleach                   3.1.0      
blis                     0.2.4      
bokeh                    1.0.4      
boto                     2.49.0     
boto3                    1.10.40    
botocore                 1.13.40    
Bottleneck               1.3.1      
branca                   0.3.1 

## Get pretrained model
need this only once

In [0]:
'''
!wget http://storage.googleapis.com/download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_11_06_2017.tar.gz
!tar -xvf faster_rcnn_resnet101_coco_11_06_2017.tar.gz
!mkdir -p "/content/drive/My Drive/model_original/faster_rcnn_resnet101_coco_11_06_2017"
!cp faster_rcnn_resnet101_coco_11_06_2017/model.ckpt.* "/content/drive/My Drive/model_original/faster_rcnn_resnet101_coco_11_06_2017/"
'''

## Minor modification
Modify python path

In [5]:
%set_env PYTHONPATH=$PYTHONPATH:models/research:models/research/slim
!echo $PYTHONPATH
!python models/research/object_detection/builders/model_builder_test.py

env: PYTHONPATH=$PYTHONPATH:models/research:models/research/slim
$PYTHONPATH:models/research:models/research/slim
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTest.test_create_experimental_model
[       OK ] ModelBuilderTest.test_create_experimental_model
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[       OK ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[       OK ] ModelBuilderTest.test_create_faster_rcnn_models_from_co

## Training
### config file
faster_rcnn_resnet101_beer.config

```
# Faster R-CNN with Resnet-101 (v1) configured for the Oxford-IIIT Pet Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  faster_rcnn {
    ### MODIFIED ###
    num_classes: 10
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_resnet101'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 14
    maxpool_kernel_size: 2
    maxpool_stride: 2
    second_stage_box_predictor {
      mask_rcnn_box_predictor {
        use_dropout: false
        dropout_keep_probability: 1.0
        fc_hyperparams {
          op: FC
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            variance_scaling_initializer {
              factor: 1.0
              uniform: true
              mode: FAN_AVG
            }
          }
        }
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.0
        iou_threshold: 0.6
        max_detections_per_class: 100
        ### MODIFIED ###
        max_total_detections: 100
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
  }
}

train_config: {
  batch_size: 1
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        manual_step_learning_rate {
          initial_learning_rate: 0.0003
          schedule {
            step: 900000
            learning_rate: .00003
          }
          schedule {
            step: 1200000
            learning_rate: .000003
          }
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  gradient_clipping_by_norm: 10.0
  ### MODIFIED ###
  fine_tune_checkpoint: "/content/drive/My Drive/model_original/faster_rcnn_resnet101_coco_11_06_2017/model.ckpt"
  from_detection_checkpoint: true
  load_all_detection_checkpoint_vars: true
  # Note: The below line limits the training process to 200K steps, which we
  # empirically found to be sufficient enough to train the pets dataset. This
  # effectively bypasses the learning rate schedule (the learning rate will
  # never decay). Remove the below line to train indefinitely.
  num_steps: 200000
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
}

train_input_reader: {
  ### MODIFIED ###
  tf_record_input_reader {
    input_path: "/content/drive/My Drive/training/beer_detection/data/train/*.tfrecord"
  }
  ### MODIFIED ###
  label_map_path: "/content/drive/My Drive/datasets/beer/tf_label_map.pbtxt"
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  num_examples: 1101
}

eval_input_reader: {
  ### MODIFIED ###
  tf_record_input_reader {
    input_path: "/content/drive/My Drive/training/beer_detection/data/eval/*.tfrecord"
  }
  ### MODIFIED ###
  label_map_path: "/content/drive/My Drive/datasets/beer/tf_label_map.pbtxt"
  shuffle: false
  num_readers: 1
}

```

In [6]:
!python models/research/object_detection/model_main.py \
    --pipeline_config_path="$TRAIN_DIR/faster_rcnn_resnet101_beer.config" \
    --model_dir="$TRAIN_DIR" \
    --num_train_steps=10000 \
    --num_eval_steps=2000 \
    --alsologtostderr


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W1229 18:40:38.459569 140358505297792 module_wrapper.py:139] From /content/models/research/object_detection/utils/config_util.py:102: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1229 18:40:38.858925 140358505297792 module_wrapper.py:139] From /content/models/research/object_detection/model_lib.py:628: The name tf.logging.warning is deprecated. Please use tf.compat.v1.logging.warning instead.

W1229 18:40:38.859267 140358505297792 model_lib.py:629] Forced number of epochs for all eval validations to be 1.

W1229 18:40:38.859490 140358505297792 module_wrapper.py:139] From /content/models/res

## Export graph

In [7]:
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path="$TRAIN_DIR/faster_rcnn_resnet101_beer.config" \
    --trained_checkpoint_prefix="$TRAIN_DIR/model.ckpt-10000" \
    --output_directory="$TRAIN_DIR/exported_graphs"


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W1229 19:20:39.231460 140180258183040 module_wrapper.py:139] From /content/models/research/object_detection/export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1229 19:20:39.240282 140180258183040 module_wrapper.py:139] From /content/models/research/object_detection/exporter.py:402: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W1229 19:20:39.243144 140180258183040 module_wrapper.py:139] From /content/models/research/object_detection/exporter.py:121: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder inst

## Check model structure
to identify input and output nodes

In [8]:
Graph = tf.GraphDef()
File = open(TRAIN_DIR + "/exported_graphs/frozen_inference_graph.pb","rb")
Graph.ParseFromString(File.read())
for Layer in Graph.node:
  print(Layer.name)


image_tensor
Cast
Preprocessor/map/Shape
Preprocessor/map/strided_slice/stack
Preprocessor/map/strided_slice/stack_1
Preprocessor/map/strided_slice/stack_2
Preprocessor/map/strided_slice
Preprocessor/map/TensorArray
Preprocessor/map/TensorArrayUnstack/Shape
Preprocessor/map/TensorArrayUnstack/strided_slice/stack
Preprocessor/map/TensorArrayUnstack/strided_slice/stack_1
Preprocessor/map/TensorArrayUnstack/strided_slice/stack_2
Preprocessor/map/TensorArrayUnstack/strided_slice
Preprocessor/map/TensorArrayUnstack/range/start
Preprocessor/map/TensorArrayUnstack/range/delta
Preprocessor/map/TensorArrayUnstack/range
Preprocessor/map/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3
Preprocessor/map/Const
Preprocessor/map/TensorArray_1
Preprocessor/map/TensorArray_2
Preprocessor/map/while/iteration_counter
Preprocessor/map/while/Enter
Preprocessor/map/while/Enter_1
Preprocessor/map/while/Enter_2
Preprocessor/map/while/Enter_3
Preprocessor/map/while/Merge
Preprocessor/map/while/Merge_

## Convert to tflite
Abort occurs...

In [9]:
converter = tf.lite.TFLiteConverter.from_frozen_graph(
  TRAIN_DIR + "/exported_graphs/frozen_inference_graph.pb",
  ["image_tensor"],
  ["detection_boxes", "detection_scores", "detection_classes", "num_detections"],
  input_shapes={"image_tensor":[1,300,300,3]}
)
tflite_model = converter.convert()


ConverterError: ignored

In [0]:
!python models/research/object_detection/export_tflite_ssd_graph.py	\
    --input_type=image_tensor \
    --pipeline_config_path="$TRAIN_DIR/faster_rcnn_resnet101_beer.config" \
    --trained_checkpoint_prefix="$TRAIN_DIR/model.ckpt-10000" \
    --output_directory="$TRAIN_DIR/exported_graphs2" \
    --add_postprocessing_op=true 
